In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.executor.cores", "4") \
    .appName("Product Recommendation") \
    .getOrCreate()

df = spark.read.csv("/home/m1nhd3n/Works/DataEngineer/product_recommendations/data/preprocess/preprocessed.csv", header=True, inferSchema=True)
df.printSchema()
df.count()

25/02/18 14:19:39 WARN Utils: Your hostname, m1nhd3n resolves to a loopback address: 127.0.1.1; using 192.168.33.129 instead (on interface wlp0s20f3)
25/02/18 14:19:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/18 14:19:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


root
 |-- fecha_dato: date (nullable = true)
 |-- ncodpers: integer (nullable = true)
 |-- ind_empleado: string (nullable = true)
 |-- pais_residencia: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- ind_nuevo: integer (nullable = true)
 |-- antiguedad: integer (nullable = true)
 |-- indrel: integer (nullable = true)
 |-- indrel_1mes: integer (nullable = true)
 |-- tiprel_1mes: string (nullable = true)
 |-- indresi: integer (nullable = true)
 |-- indext: integer (nullable = true)
 |-- canal_entrada: string (nullable = true)
 |-- indfall: integer (nullable = true)
 |-- cod_prov: integer (nullable = true)
 |-- ind_actividad_cliente: integer (nullable = true)
 |-- renta: double (nullable = true)
 |-- segmento: string (nullable = true)
 |-- ind_ahor_fin_ult1: integer (nullable = true)
 |-- ind_aval_fin_ult1: integer (nullable = true)
 |-- ind_cco_fin_ult1: integer (nullable = true)
 |-- ind_cder_fin_ult1: integer (nullable = true)
 |-- 

13431393

In [2]:
from pyspark.sql.functions import col, sum

df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

25/02/18 00:27:57 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+--------+------------+---------------+----+---+---------+----------+------+-----------+-----------+-------+------+-------------+-------+--------+---------------------+-----+--------+-----------------+-----------------+----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+---------------+-----------------+---------------+
|fecha_dato|ncodpers|ind_empleado|pais_residencia|sexo|age|ind_nuevo|antiguedad|indrel|indrel_1mes|tiprel_1mes|indresi|indext|canal_entrada|indfall|cod_prov|ind_actividad_cliente|renta|segmento|ind_ahor_fin_ult1|ind_aval_fin_ult1|ind_cco_fin_ult1|ind_cder_fin_ult1|ind_cno_fin_ult1|ind_ctju_fin_ult1|ind_ctma_fin_ult1|ind_ctop_fin_ult1|ind_ctpp_fin_ult1|ind_deco_fin_ult1|ind_deme_

In [3]:
df.show()

+----------+---------+------------+---------------+----+---+----------+---------+----------+------+-----------+-----------+-------+------+--------+-------------+-------+--------+---------------------+------------------+------------------+-----------------+-----------------+----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+---------------+-----------------+---------------+
|fecha_dato| ncodpers|ind_empleado|pais_residencia|sexo|age|fecha_alta|ind_nuevo|antiguedad|indrel|indrel_1mes|tiprel_1mes|indresi|indext|conyuemp|canal_entrada|indfall|cod_prov|ind_actividad_cliente|             renta|          segmento|ind_ahor_fin_ult1|ind_aval_fin_ult1|ind_cco_fin_ult1|ind_cder_fin_ult1|ind_cno_fin_ult1|ind_ctju_fi

In [2]:
df = df.replace(["S", "N"], ["1", "0"], subset=["indresi", "indext"])
df = df.withColumns({
    "indresi": df.indresi.cast("int"),
    "indext": df.indext.cast("int")
})
df.show()

25/02/18 14:20:38 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+--------+------------+---------------+----+---+---------+----------+------+-----------+-----------+-------+------+-------------+-------+--------+---------------------+------------------+------------------+-----------------+-----------------+----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+---------------+-----------------+---------------+
|fecha_dato|ncodpers|ind_empleado|pais_residencia|sexo|age|ind_nuevo|antiguedad|indrel|indrel_1mes|tiprel_1mes|indresi|indext|canal_entrada|indfall|cod_prov|ind_actividad_cliente|             renta|          segmento|ind_ahor_fin_ult1|ind_aval_fin_ult1|ind_cco_fin_ult1|ind_cder_fin_ult1|ind_cno_fin_ult1|ind_ctju_fin_ult1|ind_ctma_fin_ult1|ind_ctop_fin_ult1

In [3]:
from pyspark.sql import Window
from pyspark.sql import functions as F


date_df = df.select("fecha_dato").distinct()
date_window = Window.orderBy(date_df.fecha_dato)
date_df = date_df.withColumn("month_idx", F.row_number().over(date_window))
date_df.show()

25/02/18 14:20:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:20:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:20:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:20:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:20:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:20:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------+---------+
|fecha_dato|month_idx|
+----------+---------+
|2015-01-28|        1|
|2015-02-28|        2|
|2015-03-28|        3|
|2015-04-28|        4|
|2015-05-28|        5|
|2015-06-28|        6|
|2015-07-28|        7|
|2015-08-28|        8|
|2015-09-28|        9|
|2015-10-28|       10|
|2015-11-28|       11|
|2015-12-28|       12|
|2016-01-28|       13|
|2016-02-28|       14|
|2016-03-28|       15|
|2016-04-28|       16|
|2016-05-28|       17|
+----------+---------+



In [4]:
df = df.join(date_df, on="fecha_dato")
df.show()

25/02/18 14:20:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:20:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:21:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:21:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:21:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:21:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------+--------+------------+---------------+----+---+---------+----------+------+-----------+-----------+-------+------+-------------+-------+--------+---------------------+-----------------+-----------------+-----------------+-----------------+----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+---------------+-----------------+---------------+---------+
|fecha_dato|ncodpers|ind_empleado|pais_residencia|sexo|age|ind_nuevo|antiguedad|indrel|indrel_1mes|tiprel_1mes|indresi|indext|canal_entrada|indfall|cod_prov|ind_actividad_cliente|            renta|         segmento|ind_ahor_fin_ult1|ind_aval_fin_ult1|ind_cco_fin_ult1|ind_cder_fin_ult1|ind_cno_fin_ult1|ind_ctju_fin_ult1|ind_ctma_fin_ult1|ind_ctop_fi

## Vectorize
### Vectorize product cols

In [6]:
from pyspark.ml.feature import VectorAssembler

input_cols = [
    'ind_ahor_fin_ult1',
    'ind_aval_fin_ult1',
    'ind_cco_fin_ult1',
    'ind_cder_fin_ult1',
    'ind_cno_fin_ult1',
    'ind_ctju_fin_ult1',
    'ind_ctma_fin_ult1',
    'ind_ctop_fin_ult1',
    'ind_ctpp_fin_ult1',
    'ind_deco_fin_ult1',
    'ind_deme_fin_ult1',
    'ind_dela_fin_ult1',
    'ind_ecue_fin_ult1',
    'ind_fond_fin_ult1',
    'ind_hip_fin_ult1',
    'ind_plan_fin_ult1',
    'ind_pres_fin_ult1',
    'ind_reca_fin_ult1',
    'ind_tjcr_fin_ult1',
    'ind_valo_fin_ult1',
    'ind_viv_fin_ult1',
    'ind_nomina_ult1',
    'ind_nom_pens_ult1',
    'ind_recibo_ult1'
]

vecAssembler = VectorAssembler(inputCols=input_cols, outputCol="product_vec")
df = vecAssembler.transform(df)
df.show()

25/02/18 14:26:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:26:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:26:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:26:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:26:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:26:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------+--------+------------+---------------+----+---+---------+----------+------+-----------+-----------+-------+------+-------------+-------+--------+---------------------+-----------------+-----------------+-----------------+-----------------+----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+---------------+-----------------+---------------+---------+--------------------+
|fecha_dato|ncodpers|ind_empleado|pais_residencia|sexo|age|ind_nuevo|antiguedad|indrel|indrel_1mes|tiprel_1mes|indresi|indext|canal_entrada|indfall|cod_prov|ind_actividad_cliente|            renta|         segmento|ind_ahor_fin_ult1|ind_aval_fin_ult1|ind_cco_fin_ult1|ind_cder_fin_ult1|ind_cno_fin_ult1|ind_ctju_fin_ult1|ind_ctma

In [8]:
df = df.drop(*input_cols)
df.show()

25/02/18 14:28:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:28:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:28:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:28:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:28:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:28:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------+--------+------------+---------------+----+---+---------+----------+------+-----------+-----------+-------+------+-------------+-------+--------+---------------------+-----------------+-----------------+---------+--------------------+
|fecha_dato|ncodpers|ind_empleado|pais_residencia|sexo|age|ind_nuevo|antiguedad|indrel|indrel_1mes|tiprel_1mes|indresi|indext|canal_entrada|indfall|cod_prov|ind_actividad_cliente|            renta|         segmento|month_idx|         product_vec|
+----------+--------+------------+---------------+----+---+---------+----------+------+-----------+-----------+-------+------+-------------+-------+--------+---------------------+-----------------+-----------------+---------+--------------------+
|2015-02-28|  545090|           N|             ES|   V| 67|        0|       120|     1|          1|          I|      1|     0|          KAT|      0|       8|                    0|         96799.17|02 - PARTICULARES|        2|      (24,[2],[1.0])|
|2015-02-28|

## Other cols

In [11]:
from pyspark.ml.feature import StringIndexer

strIndexer = StringIndexer(inputCol="sexo", outputCol="sexo_bin")
model = strIndexer.fit(df)
df = model.transform(df)
df.drop("sexo")
df.show()

25/02/18 14:33:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:33:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:33:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:33:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:33:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:33:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------+--------+------------+---------------+----+---+---------+----------+------+-----------+-----------+-------+------+-------------+-------+--------+---------------------+-----------------+-----------------+---------+--------------------+--------+
|fecha_dato|ncodpers|ind_empleado|pais_residencia|sexo|age|ind_nuevo|antiguedad|indrel|indrel_1mes|tiprel_1mes|indresi|indext|canal_entrada|indfall|cod_prov|ind_actividad_cliente|            renta|         segmento|month_idx|         product_vec|sexo_bin|
+----------+--------+------------+---------------+----+---+---------+----------+------+-----------+-----------+-------+------+-------------+-------+--------+---------------------+-----------------+-----------------+---------+--------------------+--------+
|2015-02-28|  545090|           N|             ES|   V| 67|        0|       120|     1|          1|          I|      1|     0|          KAT|      0|       8|                    0|         96799.17|02 - PARTICULARES|        2|      (

In [15]:
from pyspark.sql.functions import log

# Apply log transformation
df = df.withColumn("renta", log(col("renta")))
df.show()

25/02/18 14:49:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:49:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:49:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:49:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:49:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:49:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------+--------+------------+---------------+----+---+---------+----------+------+-----------+-----------+-------+------+-------------+-------+--------+---------------------+------------------+-----------------+---------+--------------------+--------+
|fecha_dato|ncodpers|ind_empleado|pais_residencia|sexo|age|ind_nuevo|antiguedad|indrel|indrel_1mes|tiprel_1mes|indresi|indext|canal_entrada|indfall|cod_prov|ind_actividad_cliente|             renta|         segmento|month_idx|         product_vec|sexo_bin|
+----------+--------+------------+---------------+----+---+---------+----------+------+-----------+-----------+-------+------+-------------+-------+--------+---------------------+------------------+-----------------+---------+--------------------+--------+
|2015-02-28|  545090|           N|             ES|   V| 67|        0|       120|     1|          1|          I|      1|     0|          KAT|      0|       8|                    0|11.480393698847744|02 - PARTICULARES|        2|   

In [16]:
from pyspark.sql.functions import percentile_approx
from pyspark.sql.functions import col

# Compute Q1 and Q3
q1, q3 = df.approxQuantile("renta", [0.25, 0.75], 0.0)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

# Filter outliers
outliers = df.filter((col("renta") < lower_bound) | (col("renta") > upper_bound))
outliers.show()
outliers.count()


25/02/18 14:50:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:50:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:51:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:51:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:51:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 14:51:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------+--------+------------+---------------+----+---+---------+----------+------+-----------+-----------+-------+------+-------------+-------+--------+---------------------+------------------+-----------------+---------+--------------------+--------+
|fecha_dato|ncodpers|ind_empleado|pais_residencia|sexo|age|ind_nuevo|antiguedad|indrel|indrel_1mes|tiprel_1mes|indresi|indext|canal_entrada|indfall|cod_prov|ind_actividad_cliente|             renta|         segmento|month_idx|         product_vec|sexo_bin|
+----------+--------+------------+---------------+----+---+---------+----------+------+-----------+-----------+-------+------+-------------+-------+--------+---------------------+------------------+-----------------+---------+--------------------+--------+
|2015-02-28|  545008|           N|             ES|   V| 52|        0|       120|     1|          1|          A|      1|     0|          KAT|      0|      28|                    1|12.813395644105539|         01 - TOP|        2|(24

615349